# Model 06 inference


## Setup

### Working directory

In [1]:
# Change the working directory to project root
from pathlib import Path
import os
ROOT_DIR = Path.cwd()
while not ROOT_DIR.joinpath("src").exists():
    ROOT_DIR = ROOT_DIR.parent
os.chdir(ROOT_DIR)

### Dependencies

In [2]:
# Imports and dependencies
import torch
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR
from torcheval.metrics import BinaryAccuracy, BinaryF1Score

from src.torch_utils import get_torch_device
import json
from dataclasses import dataclass
from typing import List, Union, Tuple, Dict
from tqdm import tqdm
import random
import numpy as np
from datetime import datetime
from sklearn.model_selection import ParameterGrid

from src.model_05 import BertCrossEncoderClassifier
from src.data import RetrievalWithShortlistDataset, RetrievalDevEvalDataset, \
    InferenceClaims, RetrievedInferenceClaims, LabelClassificationDataset
from src.logger import SimpleLogger

TORCH_DEVICE = get_torch_device()

/opt/homebrew/Caskroom/miniconda/base/envs/comp90042_project/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch device is 'mps'


### File paths

In [3]:
MODEL_PATH = ROOT_DIR.joinpath("./result/models/*")
DATA_PATH = ROOT_DIR.joinpath("./data/*")
LOG_PATH = ROOT_DIR.joinpath("./result/logs/*")
RETRIEVAL_PATH = ROOT_DIR.joinpath("./result/pipeline/retrieval_classif/*")
OUTPUT_PATH = ROOT_DIR.joinpath("./result/pipeline/final_classif/*")

run_time = datetime.now().strftime('%Y_%m_%d_%H_%M')

## Load model

Use a blank pre-trained

In [4]:
# model = BertCrossEncoderClassifier(
#     pretrained_name="bert-base-uncased",
#     n_classes=3,
#     device=TORCH_DEVICE
# )

Or load fine-tuned

In [5]:
MODEL_SAVE_PATH = MODEL_PATH.with_name("model_06_bert_base_uncased_cross_encoder_label_2023_05_08_15_13.pth")
with open(MODEL_SAVE_PATH, mode="rb") as f:
    model = torch.load(f, map_location=TORCH_DEVICE)

## Inference run code

In [6]:
def inference_run(
    claims_retrieval_path:Path,
    evidence_path:Path,
    save_path:Path = None,
    batch_size:int = 64
):
    # Label map
    LABEL_MAP = {
        0: "REFUTES",
        1: "NOT_ENOUGH_INFO",
        2: "SUPPORTS",
        3: "DISPUTED"
    }
    
    # Generate claims iterations
    inference_claims = RetrievedInferenceClaims(claims_path=claims_retrieval_path)
    
    # Cumulator
    with open(claims_retrieval_path, mode="r") as f:
        claim_predictions = json.load(f)
    
    for claim_id, claim_text, evidence_ids in tqdm(inference_claims, desc="claims"):
        
        # Generate claims-evidence inference interations
        infer_data = LabelClassificationDataset(
            claim_id=claim_id,
            claims_paths=[claims_retrieval_path],
            evidence_path=evidence_path,
            training=False,
            verbose=False
        )
        infer_dataloader = DataLoader(
            dataset=infer_data,
            shuffle=False,
            batch_size=batch_size
        )
        
        print(f"running inference for {claim_id} n={len(infer_data)}")
    
        # Set model mode to evaluation
        model.eval()
        
        for batch in infer_dataloader:
            claim_texts, evidence_texts, labels, batch_claim_ids, batch_evidence_ids = batch
            texts = list(zip(claim_texts, evidence_texts))
            
            # Forward
            output, logits, seq = model(
                texts=texts,
                normalize_text=True,
                max_length=512,
                dropout=None
            )
            
            # Prediction
            predicted = torch.argmax(output, dim=1).cpu()
            
            # Apply label prediction rules, default is NEI
            if 2 in predicted and 0 in predicted:
                predicted_class = LABEL_MAP.get(3) # DISPUTED
            
            elif 2 in predicted:
                predicted_class = LABEL_MAP.get(2) # SUPPORTS
                
            elif 0 in predicted:
                predicted_class = LABEL_MAP.get(0) # REFUTES
            else:
                predicted_class = LABEL_MAP.get(1) # NOT_ENOUGH_INFO

            claim_predictions[claim_id]["claim_label"] = predicted_class
            print(f"class={predicted_class}, labels={predicted}")
            continue
    
        continue
    
    with open(save_path, mode="w") as f:
        json.dump(obj=claim_predictions, fp=f)
        print(f"saved to: {save_path}")
        
    return

## Dev inference

In [7]:
inference_run(
    claims_retrieval_path=RETRIEVAL_PATH.with_name("model_05_bert_base_2023_05_08_17_06_dev_shortlist_max_500_no_rel.json"),
    evidence_path=DATA_PATH.with_name("evidence.json"),
    save_path=OUTPUT_PATH.with_name("model_05_bert_base_2023_05_08_17_06_dev_shortlist_max_500_no_rel_final.json"),
    batch_size=24
)

loaded inference claims n=154


claims:   0%|          | 0/154 [00:00<?, ?it/s]

Torch device is 'mps'
generated dataset n=5
running inference for claim-752 n=5


claims:   1%|          | 1/154 [00:00<02:31,  1.01it/s]

class=SUPPORTS, labels=tensor([2, 2, 1, 1, 1])
Torch device is 'mps'


...

generated dataset n=5
running inference for claim-1426 n=5
class=SUPPORTS, labels=tensor([2, 2, 2, 2, 2])
Torch device is 'mps'


claims:  99%|█████████▉| 153/154 [01:51<00:00,  1.36it/s]

generated dataset n=5
running inference for claim-698 n=5
class=SUPPORTS, labels=tensor([2, 1, 1, 2, 2])
Torch device is 'mps'


claims: 100%|██████████| 154/154 [01:51<00:00,  1.38it/s]

generated dataset n=5
running inference for claim-1021 n=5
class=SUPPORTS, labels=tensor([2, 2, 2, 2, 2])
saved to: /Users/johnsonzhou/git/comp90042-project/result/pipeline/final_classif/model_05_bert_base_2023_05_08_17_06_dev_shortlist_max_500_no_rel_final.json
